In [1]:
import pickle
import pandas as pd
import numpy as np

In [6]:
df = pd.read_pickle("BNLCosmic3_DF.pkl")

ped_dict = pd.read_pickle("Ped_BNL.pkl")

In [7]:
ped_df = pd.DataFrame(ped_dict)

# Ensure "channel" column exists
ped_df["channel"] = ped_df.index

# Drop any overlapping columns from ped_df (except "channel")
overlapping_cols = set(df.columns).intersection(ped_df.columns) - {"channel"}
ped_df = ped_df.drop(columns=overlapping_cols)

# Merge pedestal data into the main DataFrame
df = pd.merge(df, ped_df, on="channel", how="left")

# Compute pedestal-corrected HG and LG
df["HG_ped_corr"] = df["HG"] - df["HGPedMean"]
df["LG_ped_corr"] = df["LG"] - df["LGPedMean"]

# Apply 3-sigma cut
df["LG_ped_corr"] = df["LG_ped_corr"].where(df["LG_ped_corr"] >= 3 * df["LGPedSigma"], 0)
df["HG_ped_corr"] = df["HG_ped_corr"].where(df["HG_ped_corr"] >= 3 * df["HGPedSigma"], 0)

# Drop pedestal mean/sigma columns now that they're no longer needed
df.drop(columns=["HGPedMean", "LGPedMean", "HGPedSigma", "LGPedSigma"], inplace=True)

print(df.head())



   event  channel    HG   LG  CAEN_brd  CAEN_ch  CAEN  HG_ped_corr  \
0      0        0   304  128         0        0     0          0.0   
1      0        1   425  142         0        1     0          0.0   
2      0        2  1180  141         0        2     0          0.0   
3      0        3   769  203         0        3     0          0.0   
4      0        4   302  154         0        4     0          0.0   

   LG_ped_corr  
0     0.000000  
1     0.000000  
2     0.000000  
3    90.635845  
4     0.000000  


In [8]:
with open('BNLCosmic3_DF.pkl', 'wb') as f:
    pickle.dump(df, f)